In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import itertools
import numpy as np


## Load labels

In [ ]:
import pandas as pd
dormer_label=pd.read_csv("\\FF_Hackathon\\Hackathon files\\dormer_labels.csv", dtype={'filename':object})
roof_label=pd.read_csv("\\FF_Hackathon\\Hackathon files\\rooftype_labels.csv", dtype={'id':object})

In [ ]:
dormer_label.head

In [ ]:
roof_label.head

In [ ]:
dormer_label.set_axis(['id', 'dormer_label'], axis='columns', inplace=True)
dormer_label.head

In [ ]:
roof_label.set_axis(['id', 'roof_label', 'labled_by'], axis='columns', inplace=True)
roof_label.head

In [ ]:
all_label = pd.merge(dormer_label, roof_label, on = "id", how = "inner")
all_label.head
# for the 327 duplicate IDs, they all should be 'no_dormers'. [investigated by Group 2]

dormer = pd.read_csv('dormer_labels.csv')
rooftype = pd.read_csv('rooftype_labels.csv')
dormer = dormer.rename(columns={'filename':'id'})
combined = pd.merge(dormer, rooftype, on=['id'], how='inner')

In [ ]:
all_label = all_label.sort_values(['dormer_label'])
all_label = all_label.drop_duplicates(subset=['id'], keep='last')

In [ ]:
all_label.head

In [ ]:
True in all_label.id.duplicated()


## Load images

In [ ]:
import os
import PIL
import tensorflow as tf
import pathlib

In [ ]:
images_dir = pathlib.Path(r"\FF_Hackathon\Hackathon files\Nadir afbeeldingen")
images_dir.as_posix()

In [ ]:
houses =list(images_dir.glob(".\\*.jpg"))

imageCount=len(houses)
imageCount

In [ ]:
im = PIL.Image.open(houses[21969])

# This method will show image in any image viewer
im.show()

## Match images with labels

In [ ]:
len(str(max(all_label.id)))



In [ ]:
len(str(min(all_label.id)))

In [ ]:
all_label = all_label.sort_values(['id'])
all_label.head
# confirmed no more duplicate IDs

In [ ]:
import glob

In [ ]:
# get only filenames instead of full paths of images
all_path_list=[os.path.basename(x) for x in glob.glob(r"\FF_Hackathon\Hackathon files\Nadir afbeeldingen\*.jpg")]



In [ ]:
df_all_path=pd.DataFrame(all_path_list)
df_all_path

In [ ]:
# remove file extensions for id matching
df_all_path[0] = df_all_path[0].str.rstrip('.jpg')
df_all_path


In [ ]:
df_all_path.rename(columns = {0:'id'}, inplace = True)
df_all_path['path_index'] = df_all_path.index
df_all_path

In [ ]:
all_label = pd.merge(all_label, df_all_path, on = "id", how = "inner")
all_label.head

In [ ]:
# add a column with full path of image for each ID
all_label['path']= "\\FF_Hackathon\\Hackathon files\\Nadir afbeeldingen\\" + all_label['id'] + ".jpg"

all_label.head

In [ ]:
all_label['path'][0]

In [ ]:
all_label.info()

In [ ]:
all_label.sample

In [ ]:
all_label["dormer_label"].unique()

In [ ]:
all_label["roof_label"].unique()

In [ ]:
all_label.to_csv('all_label.csv')
# for examination of data and storing interim result

## extract paths according to labels

In [ ]:


dormer_list=all_label[all_label['dormer_label']=="dormers"]['path']
nodormer_list=all_label[all_label['dormer_label']=="no_dormers"]['path']

# combinatie: 80
# dwarskap: 39
# langskap: 733
# platdak: 26
# schilddak: 3


langskap_list=all_label[all_label['roof_label']=="langskap"]['path']
combinatie_list=all_label[all_label['roof_label']=="combinatie"]['path']
dwarskap_list=all_label[all_label['roof_label']=="dwarskap"]['path']
platdak_list=all_label[all_label['roof_label']=="platdak"]['path']
schilddak_list=all_label[all_label['roof_label']=="schilddak"]['path']



In [ ]:
dormer_list.head

In [ ]:
nodormer_list.head

In [ ]:
langskap_list.head

In [ ]:
combinatie_list.head

In [ ]:
dwarskap_list.head

In [ ]:
platdak_list.head

In [ ]:
schilddak_list.head

## copy valid files to another folder

In [ ]:
all_label.index.size

In [ ]:
# https://stackoverflow.com/questions/33625931/copy-file-with-pathlib-in-python

import pathlib
import shutil

for i in range(all_label.index.size):
    from_file = pathlib.Path(all_label.path[i])
    to_file = pathlib.Path(r'\FF_Hackathon\Group3_1\ValidImage')
    shutil.copy(from_file, to_file)  # For Python 3.8+


In [ ]:
# add a column with new full path of image for each ID
all_label['new_path']= "\\FF_Hackathon\\Group3_1\\ValidImage\\" + all_label['id'] + ".jpg"

all_label.head

In [ ]:
all_label.new_path[0]

In [ ]:
all_label.new_path[882]

## Build Dataset

In [ ]:
#import tensorflow as tf
from tensorflow import keras
#import pandas as pd

In [ ]:
#https://stackoverflow.com/questions/62716533/mapping-images-and-their-labels
#https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence